In [1]:
!pip install fastparquet pymorphy2

  Obtaining dependency information for fastparquet from https://files.pythonhosted.org/packages/57/fd/79d39d046c71278f5b7bb0e155a762b913bea987cb3dd88d7d998956b3f0/fastparquet-2023.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.1 MB/s eta 0:00:00
  Obtaining dependency information for cramjam>=2.3 from https://files.pythonhosted.org/packages/30/79/abc388ed246721b7d56ef4410f07c8f81b1df5660f23f9b743cc6958ecdc/cramjam-2.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 55.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.6 MB/s eta 0:00:00


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sturub/automarkup.parquet
/kaggle/input/sturub/sample_submission.csv
/kaggle/input/sturub/videos.parquet
/kaggle/input/sturub/manualmarkup.csv
/kaggle/input/sturub/features.parquet
/kaggle/input/sturub/Rutube.xlsx
/kaggle/input/sturub/requirements.txt
/kaggle/input/sturub/baseline.ipynb
/kaggle/input/sturub/features/features_nov.parquet
/kaggle/input/sturub/features/features_sep.parquet
/kaggle/input/sturub/features/features_may.parquet
/kaggle/input/sturub/features/features_jun.parquet
/kaggle/input/sturub/features/features_aug.parquet
/kaggle/input/sturub/features/features_oct.parquet
/kaggle/input/sturub/features/features_jul.parquet
/kaggle/input/sturub/test_data/test_data/sample_submission.csv
/kaggle/input/sturub/test_data/test_data/submission_queries.csv
/kaggle/input/sturub/metric/metric/sample_submission.csv
/kaggle/input/sturub/metric/metric/script.py
/kaggle/input/sturub/metric/metric/ground_truth.csv


In [3]:
import math
import pyarrow.parquet as pq

In [4]:
seed = 42
np.random.seed(seed)

In [5]:
candidates = pd.read_parquet(
    '/kaggle/input/sturub/videos.parquet',
    engine='fastparquet',
    columns=['video_id', 'video_title']
)

In [6]:
candidates = candidates.sample(n=3_000_000, replace=False, random_state=seed)

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words("russian"))

def lemmatize_string(text):
    tokens = word_tokenize(text)    
    
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    lemmed_tokens = [morph.normal_forms(token)[0] for token in filtered_tokens]
    
    return ' '.join(lemmed_tokens).strip()

In [8]:
candidates['video_title'] = candidates['video_title'].apply(lemmatize_string)

In [7]:
corpus = candidates['video_title'].apply(lambda x: x.lower()).values
video_ids = candidates['video_id'].values
#del candidates

In [9]:
len(corpus)

3000000

In [10]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string

def get_tokens(docs, preprocess=preprocess_string, verbose=10000):
    
    for i, doc in enumerate(docs):
        yield preprocess(doc)
        
        # print progress if needed
        if verbose > 0 and (i + 1) % verbose == 0:
            print(f"Progress: {i + 1}")

In [11]:
from gensim.models.fasttext import FastText

ft_model = FastText(
    sg=1, # use skip-gram: usually gives better results
    #size=100, # embedding dimension (default)
    window=10, # window size: 10 tokens before and 10 tokens after to get wider context
    min_count=10, # only consider tokens with at least 10 occurrences in the corpus
    negative=15, # negative subsampling: bigger than default to sample negative examples more
    min_n=2, # min character n-gram
    max_n=5 # max character n-gram
)

In [12]:
%%time
ft_model.build_vocab(get_tokens(corpus, verbose=100000))

Progress: 100000
Progress: 200000
Progress: 300000
Progress: 400000
Progress: 500000
Progress: 600000
Progress: 700000
Progress: 800000
Progress: 900000
Progress: 1000000
Progress: 1100000
Progress: 1200000
Progress: 1300000
Progress: 1400000
Progress: 1500000
Progress: 1600000
Progress: 1700000
Progress: 1800000
Progress: 1900000
Progress: 2000000
Progress: 2100000
Progress: 2200000
Progress: 2300000
Progress: 2400000
Progress: 2500000
Progress: 2600000
Progress: 2700000
Progress: 2800000
Progress: 2900000
Progress: 3000000
CPU times: user 2min 55s, sys: 190 ms, total: 2min 55s
Wall time: 2min 55s


In [13]:
%%time
epochs = 3

for epoch in range(epochs):
    print(f"Epoch {epoch}")
    
    ft_model.train(
        get_tokens(corpus, verbose=100000),
        epochs=1,
        total_examples=ft_model.corpus_count, 
        total_words=ft_model.corpus_total_words)

Epoch 0
Progress: 100000
Progress: 200000
Progress: 300000
Progress: 400000
Progress: 500000
Progress: 600000
Progress: 700000
Progress: 800000
Progress: 900000
Progress: 1000000
Progress: 1100000
Progress: 1200000
Progress: 1300000
Progress: 1400000
Progress: 1500000
Progress: 1600000
Progress: 1700000
Progress: 1800000
Progress: 1900000
Progress: 2000000
Progress: 2100000
Progress: 2200000
Progress: 2300000
Progress: 2400000
Progress: 2500000
Progress: 2600000
Progress: 2700000
Progress: 2800000
Progress: 2900000
Progress: 3000000
Epoch 1
Progress: 100000
Progress: 200000
Progress: 300000
Progress: 400000
Progress: 500000
Progress: 600000
Progress: 700000
Progress: 800000
Progress: 900000
Progress: 1000000
Progress: 1100000
Progress: 1200000
Progress: 1300000
Progress: 1400000
Progress: 1500000
Progress: 1600000
Progress: 1700000
Progress: 1800000
Progress: 1900000
Progress: 2000000
Progress: 2100000
Progress: 2200000
Progress: 2300000
Progress: 2400000
Progress: 2500000
Progress: 26

In [14]:
%%time
ft_model.wv.most_similar("ржач", topn=10)

CPU times: user 27.4 ms, sys: 22 ms, total: 49.4 ms
Wall time: 30.3 ms


[('ренты', 0.836959958076477),
 ('коменты', 0.829413115978241),
 ('аргументы', 0.8198832869529724),
 ('сегменты', 0.8169484734535217),
 ('агенты', 0.8070856928825378),
 ('алименты', 0.7995253801345825),
 ('тенты', 0.7939766049385071),
 ('реагенты', 0.7669273018836975),
 ('ферменты', 0.7641869783401489),
 ('понты', 0.7590990662574768)]

In [15]:
ft_model.save('fasttext.model')

In [16]:
def expand_query(query, wv, topn=10):
    expanded_query = [t for t in query]
    
    for t in query:
        expanded_query.extend(s for s, f in wv.most_similar(t, topn=topn))
        
    return expanded_query

In [17]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
import numpy as np

In [18]:
%%time
corpus = [doc.lower().split() for doc in corpus]
dictionary = Dictionary(corpus)

CPU times: user 6min 48s, sys: 30.2 s, total: 7min 18s
Wall time: 7min 18s


In [19]:
%%time
bm25_model = OkapiBM25Model(dictionary=dictionary)
bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]
bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(corpus), num_terms=len(dictionary),
                                   normalize_queries=False, normalize_documents=False)

CPU times: user 1min 44s, sys: 1.14 s, total: 1min 45s
Wall time: 1min 45s


In [55]:
%%time
tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')  # Enforce binary weighting of queries

def get_top_n(query, n=5):
    
    tfidf_query = tfidf_model[dictionary.doc2bow(query)]
    similarities = bm25_index[tfidf_query]

    # get indices of top N scores
    idx = np.argpartition(similarities, -n)[-n:]
    print(idx)

    # sort top N scores and return their indices
    topn = idx[np.argsort(-similarities[idx])]
    print(topn)

    for i in topn:
        print(candidates.loc[candidates['video_id']==video_ids[i]]['video_title'].values[0])
#        print(' '.join(corpus[i]))

CPU times: user 2.44 s, sys: 122 ms, total: 2.56 s
Wall time: 2.57 s


In [56]:
%%time
query = ['однажды', 'мэр']
#экстрасенсы битва сильнейших

test_query_exp = expand_query(query, ft_model.wv)

get_top_n(test_query_exp, 5)

[1092004 2813150 2775657  377060  295262]
[ 295262 2775657  377060 2813150 1092004]
Однажды в России: Переезд к мэру
Однажды в России: Мэр разгадывает загадки
Однажды в России: Мэр заболел
Однажды в России: Лагерь мэра
Собянин боится критики. Депутатам запретили задавать мэру неудобные вопросы
CPU times: user 5.65 s, sys: 342 ms, total: 6 s
Wall time: 5.59 s


In [57]:
%%time
test_query_exp = expand_query(['спинер'], ft_model.wv)

get_top_n(test_query_exp, 5)

[ 275197  337546  942257 2645335 2999688]
[2999688 2645335  942257  337546  275197]
Дорогой спинер vs Дешовый спиннер
Лего спиннер кнопка Ютуб! 3 КРУТЫХ СПИННЕРА ЧАСТЬ 2!
Быстрая Разминка для Спины и Плеч
DIY: КАК СДЕЛАТЬ СПИННЕР ИЗ ТРАКТОРНОГО ПОДШИПНИКА!│Spinner,спинер, как сделать спинер
КАК ВЫПРЯМИТЬ ПОЗВОНОЧНИК  | УПРАЖНЕНИЯ ДЛЯ СПИНЫ
CPU times: user 5.58 s, sys: 323 ms, total: 5.9 s
Wall time: 5.52 s


In [34]:
%%time
test_query_exp = expand_query(['моя фамилия шилов'], ft_model.wv)

get_top_n(test_query_exp, 5)

[2796423  919644 1487408 2606793 1753470]
[2606793 1487408 1753470  919644 2796423]
гумилев лев.mp4
28 панфиловцев
халилов колган
куприянов петр владимирович.mp4
художник леонид герасимов
CPU times: user 323 ms, sys: 291 ms, total: 614 ms
Wall time: 259 ms
